In [44]:
%load_ext autoreload
%autoreload 2

from lstcondensation.model import LSGraphTCN

from pytorch_lightning import Trainer

from gnn_tracking.metrics.losses import PotentialLoss, BackgroundLoss
import torch
from functools import partial
from gnn_tracking.training.tc import TCModule

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'lstcondensation'

## 1. Configure data

In [31]:
model = LSGraphTCN(
    node_indim=9, edge_indim=3, h_dim=128, e_dim=128, h_outdim=12, L_hc=3
)

/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:196: UserWarning: Attribute 'hc_in' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['hc_in'])`.
  rank_zero_warn(


## 3. Configure loss functions and weights

In [35]:
from gnn_tracking.postprocessing.dbscanscanner import DBSCANHyperParamScanner


def n_trials(epoch: int) -> int:
    # if epoch < 10:
    #     return 0
    if epoch % 3 == 0:
        return 6
    else:
        return 0


# TC for track condensation
lmodel = TCModule(
    model=model,
    potential_loss=PotentialLoss(
        radius_threshold=1.0,
    ),
    background_loss=BackgroundLoss(),
    lw_repulsive=1.0,
    lw_background=0.1,
    optimizer=partial(torch.optim.Adam, lr=7.5e-4),
    cluster_scanner=DBSCANHyperParamScanner(
        n_trials=n_trials, n_jobs=3, min_samples_range=(1, 1)
    ),
)

## 4. Train the model

In [36]:
from pytorch_lightning.loggers import WandbLogger

wl = WandbLogger(
    project="lst_oc",
    group="first",
    offline=True,
)

/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/pytorch_lightning/loggers/wandb.py:396: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(


In [43]:
from lstcondensation.loader import default_data_module

trainer = Trainer(
    accelerator="gpu", log_every_n_steps=1, logger=wl, enable_progress_bar=False
)
trainer.fit(model=lmodel, datamodule=default_data_module)

ModuleNotFoundError: No module named 'lstcondensation'

In [39]:
print("test")

test


In [40]:
! pwd

/home/kl5675/Documents/23/git_sync/hpo/slurm
